In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import os
import warnings
from machine_learning.utils import split_data, scale_dataset, get_distribution, plot_distribution
from machine_learning.neural_networks.utils import plot_history
from machine_learning.neural_networks.shallow_fnn import train_shallow_fnn_model
from configs.enums import Column
import numpy as np
from configs.data import MACHINE_LEARNING_DATASET_PATH, MODELS_PATH, VERSION
import shap
import tensorflow as tf
import machine_learning.neural_networks.learning_rate_schedulers as lrs
from typing import Tuple

warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(0)

## Loading the dataset

In [ ]:
df = pd.read_excel(MACHINE_LEARNING_DATASET_PATH)
train_df, valid_df, test_df = split_data(df)
train, x_train, train_labels = scale_dataset(train_df, oversample=True)
valid, x_val, val_labels = scale_dataset(valid_df, oversample=False)
test, x_test, test_labels = scale_dataset(test_df, oversample=False)

print(f"Train: {x_train.shape}, Valid: {x_val.shape}, Test: {x_test.shape}")

## Utility function definitions

In [ ]:
def print_results(model) -> Tuple:
    y_pred_train = model.predict(x_train).argmax(axis=1)
    print("\n###### Training ######")
    print(classification_report(train_labels, y_pred_train))
    
    y_pred_valid = model.predict(x_val).argmax(axis=1)
    print("\n###### Validation ######")
    print(classification_report(val_labels, y_pred_valid))
    
    y_pred = model.predict(x_test).argmax(axis=1)
    print("\n###### Test ######")
    print(classification_report(test_labels, y_pred))
    
    return y_pred_train, y_pred_valid, y_pred

## Load model from file

In [ ]:
model_file = "RawData.9c_Adam_1024_0_#FactorScheduler-factor_0.995-stop_factor_0.00075-base_lr_0.00075#_200_25_32_0.19385148584842682.shallow_fnn.keras"
model = tf.keras.models.load_model(os.path.join(MODELS_PATH, "1e-07_09_0999_0_None_None", model_file))

_, _, y_pred = print_results(model)

## Tuning

### Parameter tuning

In [ ]:
def tune_model(df,
               units,
               dropout_rates,
               learning_rates,
               epochs=200,
               patience=[10, 20],
               batch_sizes=[128]):
    import time
    from datetime import timedelta
    
    least_val_lost_file_name = f"tuning_least_val_loss.fnn.keras"
    
    to_hh_mm_ss = lambda seconds: str(timedelta(seconds=seconds)).rsplit(".")[0]
    
    least_val_loss = float('inf')
    least_val_loss_params = []
    least_val_loss_accuracy = float('inf')
    least_val_loss_model = None  
    least_val_loss_history = None
    eta = None
    
    time_past = 0
    
    i = 1 
    max = len(units) * len(dropout_rates) * len(learning_rates) * len(patience) * len(batch_sizes)
    
    print("[prev: N/A] [eta: TBD]")
    
    for u in units:
        for dr in dropout_rates:
            for lr in learning_rates: 
                for pt in patience:
                    for bt in batch_sizes:
                        start_time = time.time()

                        print(f"[{i}/{max}] Units: {u}; Dropout rate: {dr}; Learning rate: {lr}; Patience: {pt}; Batch size: {bt}")
                        
                        # TODO: add batch_size param to tune
                        model, history, num_classes = train_shallow_fnn_model(
                            df, 
                            epochs=epochs, 
                            patience=pt, 
                            units=u,
                            dropout_rate=dr,
                            learning_rate=lr,
                            epsilon = 1e-07,
                            beta_1 = 0.9,
                            beta_2 = 0.999,
                            weight_decay = 0,
                            clipnorm = None,
                            clipvalue = None,
                            batch_size=bt,
                            verbose=0,
                            disable_save=True,
                            disable_plot_history=True,
                            disable_print_report=True)
                        
                        val_loss, val_acc = model.evaluate(x_test, test_labels)
                        print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                        if val_loss < least_val_loss:
                            model.save(os.path.join(MODELS_PATH, least_val_lost_file_name))
                            least_val_loss = val_loss
                            least_val_loss_params = [ u, dr, lr, pt, bt]
                            least_val_loss_accuracy = val_acc
                            least_val_loss_model = model
                            least_val_loss_history = history
                            
                        duration = time.time() - start_time
                        time_past += duration
                        avg_duration = time_past / i
                        eta = time_past + avg_duration * (max - i)
                        
                        print(f"\n[eta: {to_hh_mm_ss(time_past)}/{to_hh_mm_ss(eta)}] [prev: {to_hh_mm_ss(duration)}] [avg: {to_hh_mm_ss(avg_duration)}]")
                            
                        i += 1
    
    u, dr, lr, pt, bt = least_val_loss_params
    print("\nLeast validation loss:")              
    print(f"\tParams:\t {{Units: {u}; Dropout rate: {dr}; Learning rate: {lr}; Patience: {pt}; Batch size: {bt}}}")
    print("\tLoss:\t", least_val_loss)
    print("\tAccuracy:\t", least_val_loss_accuracy)
    
    best_model_file_name = f"{VERSION}_Adam_{u}_{dr}_{lr}_{epochs}_{pt}_{bt}_{least_val_loss}.shallow_fnn.keras"
    os.rename(
        os.path.join(MODELS_PATH, least_val_lost_file_name), 
        os.path.join(MODELS_PATH, best_model_file_name))
    print(f"\nModel has been saved as '{best_model_file_name}'")
    
    plot_history(least_val_loss_history, num_classes)
    
    print_results(least_val_loss_model)
    
    return least_val_loss_model, least_val_loss_history, num_classes


In [ ]:
model, history, _ = tune_model(
    df=df,
    units=[512, 768, 1024],
    dropout_rates=[0],
    learning_rates=[
        0.0015, 
        0.00175, 
        lrs.FactorScheduler(factor=0.995, stop_factor=0.00075, base_lr=0.002), 
        lrs.FactorScheduler(factor=1.005, stop_factor=0.002, base_lr=0.00075)],
    patience=[10, 20, 25, 30], 
    batch_sizes=[32, 64, 128])


### Hyperparameter tuning

In [ ]:
def tune_hyperparameters(df,
                         learning_rates,
                         epsilons = [1e-07],
                         beta_1s = [0.9],
                         beta_2s = [0.999],
                         weight_decay = [None],
                         clipnorm = [None],
                         clipvalue = [None],
                         patience=[10, 20]):
    import time
    from datetime import timedelta
    
    least_val_lost_file_name = f"tuning_least_val_loss.fnn.keras"
    
    to_hh_mm_ss = lambda seconds: str(timedelta(seconds=seconds)).rsplit(".")[0]
    
    least_val_loss = float('inf')
    least_val_loss_params = []
    least_val_loss_accuracy = float('inf')
    least_val_loss_model = None  
    least_val_loss_history = None
    eta = None
    
    time_past = 0
    
    i = 1 
    max = len(epsilons) * len(beta_1s) * len(beta_2s) * len(learning_rates) * len(weight_decay) * len(clipnorm) * len(clipvalue) * len(patience)
    
    print("[prev: N/A] [eta: TBD]")
    
    for e in epsilons:
        for b1 in beta_1s:
            for b2 in beta_2s: 
                for lr in learning_rates: 
                    for wd in weight_decay:
                        for cn in clipnorm:
                            for cv in clipvalue:
                                for pt in patience:
                                    start_time = time.time()
            
                                    print(f"[{i}/{max}] Epsilons: {e}; Beta 1: {b1}; Beta 2: {b2}; Learning rate: {lr}; Weight decay: {wd}; Clipnorm: {cn}; Clipvalue: {cv}; Patience: {pt}")
                                    
                                    # TODO: add batch_size param to tune
                                    model, history, num_classes = train_shallow_fnn_model(
                                        df, 
                                        epochs=1000, 
                                        patience=pt, 
                                        units=1024,
                                        dropout_rate=0,
                                        learning_rate=lr,
                                        epsilon = e,
                                        beta_1 = b1,
                                        beta_2 = b2,
                                        weight_decay = wd,
                                        clipnorm = cn,
                                        clipvalue = cv,
                                        batch_size=32,
                                        verbose=0,
                                        disable_save=True,
                                        disable_plot_history=True,
                                        disable_print_report=True)
                                    
                                    val_loss, val_acc = model.evaluate(x_test, test_labels)
                                    print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                                    if val_loss < least_val_loss:
                                        model.save(os.path.join(MODELS_PATH, least_val_lost_file_name))
                                        least_val_loss = val_loss
                                        least_val_loss_params = [e, b1, b2, lr, wd, cn, cv, pt]
                                        least_val_loss_accuracy = val_acc
                                        least_val_loss_model = model
                                        least_val_loss_history = history
                                        
                                    duration = time.time() - start_time
                                    time_past += duration
                                    avg_duration = time_past / i
                                    eta = time_past + avg_duration * (max - i)
                                    
                                    print(f"\n[eta: {to_hh_mm_ss(time_past)}/{to_hh_mm_ss(eta)}] [prev: {to_hh_mm_ss(duration)}] [avg: {to_hh_mm_ss(avg_duration)}]")
                                        
                                    i += 1
    
    e, b1, b2, lr, wd, cn, cv, pt = least_val_loss_params
    print("\nLeast validation loss:")              
    print(f"\tParams:\t {{Epsilons: {e}; Beta 1: {b1}; Beta 2: {b2}; Learning rate: {lr}; Weight decay: {wd}; Clipnorm: {cn}; Clipvalue: {cv}; Patience: {pt}}}")
    print("\tLoss:\t", least_val_loss)
    print("\tAccuracy:\t", least_val_loss_accuracy)
    
    best_model_file_name = f"{VERSION}_Adam_hyper_{e}_{b1}_{b2}_{lr}_{wd}_{cn}_{cv}_{pt}_{least_val_loss}.shallow_fnn.keras"
    os.rename(
        os.path.join(MODELS_PATH, least_val_lost_file_name), 
        os.path.join(MODELS_PATH, best_model_file_name))
    print(f"\nModel has been saved as '{best_model_file_name}'")
    
    plot_history(least_val_loss_history, num_classes)
    
    print_results(least_val_loss_model)
    
    return least_val_loss_model, least_val_loss_history, num_classes


In [ ]:
model, history, _ = tune_hyperparameters(
    df=df,
    learning_rates=[lrs.FactorScheduler(factor=0.995, stop_factor=0.00075, base_lr=0.002), ], 
    epsilons=[1e-06, 1e-07, 1e-08],
    beta_1s=[0.4, 0.8, 0.9], # CANNOT BE EQUAL OR HIGHER THAN 1
    beta_2s=[0.9, 0.999], # CANNOT BE EQUAL OR HIGHER THAN 1
    weight_decay=[None, 0.01],
    clipnorm=[None],
    clipvalue=[None],
    patience=[25]
)

### Manual training

In [ ]:
model, _, _ = train_shallow_fnn_model(
                        df, 
                        epochs=2000, 
                        patience=25, 
                        units=1024,
                        dropout_rate=0,
                        learning_rate=lrs.FactorScheduler(factor=0.995, stop_factor=0.00075, base_lr=0.002),
                        verbose=2,
                        epsilon=1e-07,
                        beta_1=0.9,
                        beta_2=0.999,
                        weight_decay=0,
                        clipnorm=None,
                        clipvalue=None,
                        disable_print_report=True,
                        disable_save=True)

_, _, y_pred = print_results(model)

## Test (y-pred) difference plotting

In [ ]:
distribution = get_distribution(test_df, y_pred)
# print(distribution)
plot_distribution(distribution)

In [ ]:
# def test():
#     import os
#     result = test_df
#     result["predicted_country_risk"] = model.predict(x_test).argmax(axis=1)
# 
#     incorrectly_predicted = result[result["country_risk"] != result["predicted_country_risk"]]
# 
#     m_df = pd.read_excel(MERGED_DATASET_PATH)
#     wm_df = m_df.iloc[incorrectly_predicted.index, ]
# 
#     wm_df["country_risk"] = map_label_to_name(incorrectly_predicted["country_risk"])
#     wm_df["predicted_country_risk"] = map_label_to_name(incorrectly_predicted["predicted_country_risk"])
# 
#     from configs.enums import RISKCLASSIFICATIONS
# 
#     classifications = RISKCLASSIFICATIONS.get_attributes()
#     def percentage_from_boundary(country_risk: str, risk_score: float) -> float:
#         classification = classifications[country_risk]
#         lower_percentage = abs(risk_score / classification.lower_bound - 1)
#         upper_percentage =  abs(risk_score / classification.upper_bound - 1)
#         return min(lower_percentage, upper_percentage)
# 
#     wm_df["percentage_from_boundary"] = [percentage_from_boundary(cr, nr) for cr, nr in zip(wm_df["country_risk"], wm_df["norm_risk"])]
# 
#     cols = ["year", "country"] + list(incorrectly_predicted.columns) + ["norm_risk", "percentage_from_boundary"]
#     wm_df = wm_df[cols]
#     wm_df["within_2.5%"] = [abs(x) <= 0.025 for x in wm_df["percentage_from_boundary"]]
#     return wm_df
# 
# t = test()
# t["percentage_from_boundary"].describe()


## Output .xlsx of incorrectly predicted rows

In [ ]:
from machine_learning.utils import output_incorrectly_predicted_xlsx
output_incorrectly_predicted_xlsx(test_df, y_pred, "shallow_fnn")  

## Shap; Feature importance

In [ ]:
explainer = shap.KernelExplainer(model.predict, x_train)
shap_values = explainer.shap_values(shap.sample(x_test, 20), nsamples=100, random_state=41) # default of nsamples = 2 * X.shape[1] + 2048 = 2066 
# explainer.save()

from configs.enums import RISKCLASSIFICATIONS
feature_names = df.columns.tolist()
feature_names.remove(Column.COUNTRY_RISK)

In [ ]:
shap.summary_plot(shap_values, x_test, 
                  feature_names=feature_names,
                  class_names=RISKCLASSIFICATIONS.get_names())

In [ ]:
shap.summary_plot(shap_values, x_test, 
                  feature_names=feature_names,
                  class_names=RISKCLASSIFICATIONS.get_names(), plot_size=(20, 10))